In [624]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar, brute, dual_annealing
import pygad
from importlib import reload
from IPython.display import clear_output
import os

import tgs 
import rgs 
import key_rate_optimization as opt

reload(tgs)
reload(rgs)
reload(opt)

<module 'key_rate_optimization' from '/Users/gordonwongyl/Documents/ORF/Reproduce_result/key_rate_optimization.py'>

(1000000.0, None, 0.0, None)